# Hand Gesture Model Training
**Kumar Shivraj 21BTRCL055**

In [5]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

# Define image height and width
img_height, img_width = 224,224

# Define batch size
batch_size = 32

# Main folder containing labeled subfolders with images
main_folder = "D:\\hand img"

# Initialize lists to store image paths and corresponding labels
image_paths = []
labels = []

# Traverse through each subfolder (class)
for label in os.listdir(main_folder):
    label_folder = os.path.join(main_folder, label)
    if os.path.isdir(label_folder):
        # Traverse through images in the current subfolder
        for image_name in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_name)
            # Append image path and corresponding label to the lists
            image_paths.append(image_path)
            labels.append(label)
# Convert labels to numeric format
label_to_index = {label: index for index, label in enumerate(set(labels))}
labels = [label_to_index[label] for label in labels]

# Split the dataset into training, validation, and testing sets
train_paths, temp_paths, train_labels, temp_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)
valid_paths, test_paths, valid_labels, test_labels = train_test_split(temp_paths, temp_labels, test_size=0.5, random_state=42)

# Function to preprocess images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = img / 255.0  # Normalize
    return img

# Load and preprocess training, validation, and testing data
train_data = np.array([preprocess_image(image_path) for image_path in train_paths])
valid_data = np.array([preprocess_image(image_path) for image_path in valid_paths])
test_data = np.array([preprocess_image(image_path) for image_path in test_paths])

# Convert labels to categorical format
num_classes = len(set(labels))  # Number of unique classes
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = to_categorical(valid_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

# Load pretrained VGG16 model (excluding the top FC layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pretrained layers
for layer in base_model.layers:
    layer.trainable = False
# Add custom FC layers on top of the pretrained layers
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout layer
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Define data augmentation for training data
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=None  # No need for preprocess_input as images are already normalized
)

# Create data generators
train_generator = train_datagen.flow(train_data, train_labels, batch_size=batch_size)

# Define callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with data augmentation and callbacks
history = model.fit(
    train_generator,
    epochs=15,
    steps_per_epoch=len(train_data) // batch_size,
    validation_data=(valid_data, valid_labels),
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(test_data, test_labels)

# Print evaluation results
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions on the test data


Epoch 1/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.3202 - loss: 3.3369 - val_accuracy: 0.9000 - val_loss: 0.3531 - learning_rate: 0.0010
Epoch 2/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 16s 209ms/step - accuracy: 0.7812 - loss: 0.3866 - val_accuracy: 0.9036 - val_loss: 0.3562 - learning_rate: 0.0010
Epoch 3/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.7152 - loss: 0.7805 - val_accuracy: 0.9679 - val_loss: 0.1424 - learning_rate: 0.0010
Epoch 4/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 17s 218ms/step - accuracy: 0.7812 - loss: 0.2862 - val_accuracy: 0.9750 - val_loss: 0.1355 - learning_rate: 0.0010
Epoch 5/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.8296 - loss: 0.5081 - val_accuracy: 0.9786 - val_loss: 0.1167 - learning_rate: 0.0010
Epoch 6/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 16s 208ms/step - accuracy: 0.8438 - loss: 0.1683 - val_accuracy: 0.9643 - val_loss: 0.1127 - learning_rate: 0.0010
Epoch 7/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.8610 - loss: 0.4004 - v

In [6]:
# Save the model
model.save("hand_gesture_model3.h5")
print("Model saved successfully.")


Model saved successfully.
